In [0]:
# data analysis packages
import pandas as pd
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Dataset")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
full_df_path = './s_rand_combined_df.csv'

full_df = pd.read_csv(full_df_path)
df = full_df.drop(columns=['Unnamed: 0'])
df.shape

(70626, 116)

In [0]:
#based on work done in, just applied to this dataset: https://github.com/vinayakumarr/Network-Intrusion-Detection/blob/master/KDDCup%2099/SimpleRNN/multiclass/lskeras.py


from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, SimpleRNN, Activation
from keras import layers
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.metrics import categorical_accuracy

from sklearn.preprocessing import Normalizer


X = df.drop(['label'],axis=1).values

Y = df['label'].values
y_binary = to_categorical(Y)


#manually doing the tran test split
X_train = X[0:56500,:]
X_test = X[56500:70626,:]

Y_train = y_binary[0:56500,:]
Y_test = y_binary[56500:70626,:]

#Transforming data
'''
scaler = Normalizer().fit(X_train)
trainX = scaler.transform(X_train)

scaler = Normalizer().fit(X_test)
trainX = scaler.transform(X_test)
'''

#reshaping the data for 3D Keras RNN format: 

X_train = X_train.reshape(56500, 1, 115)
X_test = X_test.reshape(14126, 1, 115)

Y_train = Y_train.reshape(56500, 1, 3)
Y_test = Y_test.reshape(14126, 1, 3)


cols = X_train.shape[2] - 1 #without labels

batch_size = 32


early_stopping_monitor = EarlyStopping(patience=5)

model = Sequential()
model.add(layers.SimpleRNN(64, input_shape=(None, 115), return_sequences=True))
'''
model.add(Dropout(0.1))
model.add(layers.Dense(5))
model.add(Dropout(0.1))
'''

model.add(layers.Dropout(0.1))
model.add(layers.SimpleRNN(64,return_sequences=True))
model.add(layers.Dropout(0.1))
model.add(layers.SimpleRNN(64, return_sequences=True)) 
model.add(layers.Dropout(0.1))


'''
model.add(layers.Dropout(0.1))
model.add(layers.SimpleRNN(64,return_sequences=True))
model.add(layers.Dropout(0.1))
model.add(layers.SimpleRNN(64, return_sequences=True))
model.add(layers.Dropout(0.1))
model.add(layers.SimpleRNN(64, return_sequences=False))
model.add(layers.Dropout(0.1))
'''

model.add(layers.Dense(3, activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='sgd',metrics=['accuracy']) #used adam before - sgd - RMSprop


#training the model
history = model.fit(X_train,Y_train, batch_size=batch_size, epochs=30, validation_data=(X_test,Y_test), callbacks=[early_stopping_monitor])

nn_pred = model.predict(X_test)


model.summary()

trainScore = model.evaluate(X_train, Y_train, verbose=0)
print(trainScore)


In [0]:
#based on, just applied to this dataset: https://github.com/naviprem/ids-deep-learning/blob/master/src/train_predict.py

X = df.drop(['label'],axis=1).values

Y = df['label'].values
y_binary = to_categorical(Y)

#manually doing the tran test split
X_train = X[0:56500,:]
X_test = X[56500:70626,:]

Y_train = y_binary[0:56500,:]
Y_test = y_binary[56500:70626,:]

#Transforming data
'''
scaler = Normalizer().fit(X_train)
trainX = scaler.transform(X_train)

scaler = Normalizer().fit(X_test)
trainX = scaler.transform(X_test)
'''

#reshaping the data for 3D Keras RNN format: 

X_train = X_train.reshape(56500, 1, 115)
X_test = X_test.reshape(14126, 1, 115)

Y_train = Y_train.reshape(56500, 1, 3)
Y_test = Y_test.reshape(14126, 1, 3)


model.add(Dense(256, activation='relu', input_shape=(42,)))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(.4))
model.add(Dense(16, activation='relu'))
model.add(Dropout(.4))
model.add(Dense(8, activation='relu'))
model.add(Dropout(.4))
model.add(Dense(3, activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

history = model.fit(X_train,Y_train, batch_size=batch_size, epochs=30, validation_data=(X_test,Y_test), callbacks=[early_stopping_monitor])

nn_pred = model.predict(X_test)

model.summary()

trainScore = model.evaluate(X_train, Y_train, verbose=0)
print(trainScore)

Train on 56500 samples, validate on 14126 samples
Epoch 1/30
56500/56500 [==============================] - 12s 217us/step - loss: 0.8601 - acc: 0.4949 - val_loss: 0.7926 - val_acc: 0.5155
Epoch 2/30
56500/56500 [==============================] - 10s 181us/step - loss: 0.8179 - acc: 0.5129 - val_loss: 0.7833 - val_acc: 0.5155
Epoch 3/30
56500/56500 [==============================] - 10s 181us/step - loss: 0.8080 - acc: 0.5202 - val_loss: 0.7767 - val_acc: 0.5155
Epoch 4/30
56500/56500 [==============================] - 10s 182us/step - loss: 0.8029 - acc: 0.5204 - val_loss: 0.7709 - val_acc: 0.5155
Epoch 5/30
56500/56500 [==============================] - 10s 182us/step - loss: 0.8006 - acc: 0.5574 - val_loss: 0.7708 - val_acc: 0.5865
Epoch 6/30
56500/56500 [==============================] - 10s 183us/step - loss: 0.7988 - acc: 0.5762 - val_loss: 0.7643 - val_acc: 0.5860
Epoch 7/30
56500/56500 [==============================] - 11s 187us/step - loss: 0.7942 - acc: 0.5762 - val_loss: 0.

https://github.com/manojkumar-github/Intrusion-Detection-System-for-IoT-networks-using-Gated-Recurrent-Neural-Networks-GRU/blob/master/GRU_SingleLayer.ipynb

https://link.springer.com/article/10.1007/s00521-010-0487-0

https://arxiv.org/pdf/1711.06798.pdf

https://github.com/google-research/morph-net

https://towardsdatascience.com/with-morphnet-google-helps-you-build-faster-and-smaller-neural-networks-586e0baf7c36